In [1]:

# Colab 셀에 이 코드를 삽입하고 실행
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# inference.ipynb

# =========================================================
# 셀 1: 필수 라이브러리 설치 및 초기 설정
# =========================================================
!pip install transformers deep-translator -q

from deep_translator import GoogleTranslator
from transformers import pipeline
import torch

MODEL_PATH = "/content/drive/MyDrive/Assignment5_Files/my_word_classifier"



In [12]:
# inference.ipynb - 셀 2 수정

def analyze_and_translate(text):

    # 저장된 모델 불러오기 (MODEL_PATH 변수 사용)
    try:
        # 👈 Drive 경로 변수 사용
        classifier = pipeline("text-classification", model=MODEL_PATH, device=-1)
    except Exception as e:
        print(f"🚨 모델 로드 오류: {e}")
        print("👉 training.ipynb 실행 및 Drive 경로를 확인하세요.")
        return

    translator = GoogleTranslator(source='ko', target='es')

    print(f"▶ 입력 문장: {text}")
    words = text.split()

    unknown_words = []
    seen_words = set()

    print("-" * 30)
    for word in words:
        clean_word = word.strip(".,?!\"'")
        if not clean_word: continue

        if any(ch.isdigit() for ch in clean_word):
            continue

        if len(clean_word) == 1:
            continue

        if clean_word in seen_words:
            continue
        seen_words.add(clean_word)

        # AI에게 물어보기
        result = classifier(clean_word, truncation=True, max_length=32)[0]
        label = result['label']

        # 'LABEL_0(모름)'이라고 판단하면 -> 번역 실행!
        if label == 'LABEL_0':
            translated = translator.translate(clean_word)
            print(f"🚨 어려운 단어 감지: '{clean_word}' -> (스페인어: {translated})")
            unknown_words.append((clean_word, translated))
        else:
            pass

    if not unknown_words:
        print("✅ 모든 단어가 '아는 단어(쉬움)'로 분류되었습니다.")
    else:
        print(f"\n=> 총 {len(unknown_words)}개의 어려운 단어를 찾았습니다.")
    print("=" * 30 + "\n")




In [14]:
import re
from collections import Counter
from transformers import pipeline
from deep_translator import GoogleTranslator

def analyze_and_translate(text):
    # 1) 모델 로드
    try:
        classifier = pipeline("text-classification", model=MODEL_PATH, device=-1)
    except Exception as e:
        print(f"🚨 모델 로드 오류: {e}")
        print("👉 training.ipynb 실행 및 Drive 경로를 확인하세요.")
        return

    translator = GoogleTranslator(source='ko', target='es')

    print(f"▶ 입력 문장:\n{text}\n")

    # 기본 전처리
    raw_words = text.split()
    # 단어별 등장 횟수(선택): 나중에 (n회 등장) 표시할 때 사용
    word_counts = Counter([w.strip(".,?!\"'") for w in raw_words if w.strip(".,?!\"'")])

    unknown_words = []          # [(ko, es, count), ...]
    seen_words = set()          # 중복 방지용

    print("-" * 40)

    for word in raw_words:
        clean_word = word.strip(".,?!\"'")   # 양쪽 기호 제거
        if not clean_word:
            continue

        # 🔹 1) 숫자 포함된 토큰은 스킵 (2023, 2025, 2027년 등)
        if any(ch.isdigit() for ch in clean_word):
            continue

        # 🔹 2) 길이 1짜리(예: '■', '·') 스킵
        if len(clean_word) == 1:
            continue

        # 🔹 3) 이미 처리한 단어면 스킵 (중복 출력 방지)
        if clean_word in seen_words:
            continue
        seen_words.add(clean_word)

        # 2) 분류 실행
        result = classifier(clean_word, truncation=True, max_length=32)[0]
        label = result["label"]

        # LABEL_0 = 어려운 단어로 가정
        if label == "LABEL_0":
            translated = translator.translate(clean_word)
            count = word_counts.get(clean_word, 1)
            unknown_words.append((clean_word, translated, count))

    # 3) 출력 형식 정리
    if not unknown_words:
        print("✅ 어려운 단어로 분류된 항목이 없습니다.")
    else:
        print("📌 어려운 단어 목록\n")
        for i, (ko, es, cnt) in enumerate(unknown_words, start=1):
            # 등장 횟수 1회는 생략하고, 2회 이상일 때만 표시
            cnt_str = f" (등장 {cnt}회)" if cnt > 1 else ""
            print(f"{i}. {ko}{cnt_str} → {es}")
        print(f"\n=> 총 {len(unknown_words)}개의 서로 다른 어려운 단어를 찾았습니다.")

    print("-" * 40 + "\n")


In [18]:
import re

def clean_korean_word(word):
    # 1) 양쪽 기호 제거
    word = word.strip(".,?!\"'“”‘’·()[]{}")

    # 2) 한글 조사 제거 (은/는/이/가/을/를/에/에서/으로/과/와/에게/한테)
    #    예: 발신자는 → 발신자 / 보도자료를 → 보도자료
    josa_pattern = r"(은|는|이|가|을|를|에|에서|으로|로|과|와|에게|한테|부터|까지|마다|조차|라도|처럼|처럼|밖에|뿐|■)"
    word = re.sub(josa_pattern + r"\s*$", "", word)

    # 3) 빈 문자열 반환 방지
    return word


def analyze_and_translate(text):
    try:
        classifier = pipeline("text-classification", model=MODEL_PATH, device=-1)
    except Exception as e:
        print(f"🚨 모델 로드 오류: {e}")
        return

    translator = GoogleTranslator(source='ko', target='es')

    print(f"▶ 입력 문장:\n{text}\n")

    words = text.split()
    seen = set()
    results = []

    print("-" * 40)
    for w in words:
        clean = clean_korean_word(w)

        # 1) 빈 문자열 / 숫자 포함 토큰 제거
        if not clean or any(ch.isdigit() for ch in clean):
            continue

        # 2) 고유 기호 단독(■ 등) 제거
        if len(clean) == 1 and not clean.isalpha():
            continue

        # 3) 중복 제거
        if clean in seen:
            continue
        seen.add(clean)

        # 4) 분류 실행
        pred = classifier(clean, truncation=True, max_length=32)[0]

        if pred["label"] == "LABEL_0":  # 어려운 단어
            translated = translator.translate(clean)

            # 번역 결과에서도 따옴표 제거
            translated = translated.strip("“”‘’\"'")

            results.append((clean, translated))

    # 결과 출력
    if not results:
        print("✅ 어려운 단어 없음\n")
        return

    print("📌 어려운 단어 목록\n")
    for i, (ko, es) in enumerate(results, 1):
        print(f"{i}. {ko} → {es}")

    print(f"\n=> 총 {len(results)}개의 어려운 단어를 찾았습니다.")
    print("-" * 40 + "\n")


In [19]:
# =========================================================
# 셀 3: 실제 예시 테스트 (예시 1)
# =========================================================
# 예시 1: 뉴스 (어려운 단어 예상)
analyze_and_translate(''' 내년 1월부터 배당소득 분리과세가 시행되면서 자본시장 과세체계가 큰 변곡점을 맞았다.

그동안 연간 금융소득 2000만원을 초과하면 종합과세로 묶이던 구조가 일부 손질되면서, 배당 투자 매력이 높아졌다는 평가가 나온다. 다만 전문가들은 이번 조치를 '세제 개편의 출발점'으로 보고 있다. 거래·보유·신규 자산을 아우르는 현행 과세체계가 국제 기준과 여전히 괴리를 보이고 있어, 후속 정비가 필수적이라는 이유에서다.

■금투세·거래세 정비 1순위

7일 정치권과 관련 당국에 따르면 지난 2일 통과된 조세특례제한법 개정안은 고배당 상장기업의 배당소득에 대해 분리과세를 적용하는 것이 핵심이다. 세율은 △2000만원 이하 14% △2000만원 초과~3억원 이하 20% △3억원 초과~50억원 이하 25% △50억원 초과 30%로 구간이 신설됐다. 적용 대상은 배당성향 40% 이상이거나, 배당성향 25% 이상이면서 전년 대비 10% 이상 배당을 늘린 상장사로, 내년 결산배당부터 바로 적용된다.

정부와 국회는 이번 조치가 기업의 배당정책을 개선하고, 주주환원 문화를 확산시키는 계기가 될 것으로 기대하고 있다. 그러나 전문가들은 "첫발을 뗐을 뿐"이라며, 구조적 문제를 해결하려면 추가적인 세제 개편이 필요하다고 지적한다.

우선 가장 시급한 과제로 꼽히는 것은 '금융투자소득세(금투세)'와 '증권거래세'의 체계 정비다. 기획재정부는 내년 1월부터 증권거래세율을 코스피 0.05%, 코스닥 0.20%로 각각 인상하기로 했다. 금투세 도입을 전제로 인하했던 증권거래세율을 금투세 도입 무산으로 인해 다시 올리는 것이다.

다만 거래 단계에서 부과되는 이 세금은 소득이 아닌 '거래 행위' 자체에 세금을 매기는 구조로, 주식시장 유동성 위축과 거래비용 증가 요인으로 작용할 수 있다는 비판이 나온다. 특히 경제협력개발기구(OECD) 주요국이 거래세를 폐지하거나 최소 수준으로 유지하고 있는 점을 고려하면, 앞으로 우리 세제가 '거래에 과세할지, 이익에 과세할지' 방향을 명확히 정립하는 것이 핵심 과제라는 지적이 나온다.

■OECD 최고 수준 상속세…지배구조 개선 걸림돌

상속·증여세 체계의 재검토도 필요하다는 지적이 나온다. 올해 정부와 국회는 상속받은 만큼 세금을 내는 유산취득세 도입을 포함한 상속세 개정안을 논의했지만 국회의 문턱을 넘지 못했다. 국회예산정책처에 따르면 OECD 회원국 가운데 전체 세수 대비 상속·증여세 비율이 가장 높은 국가는 한국(1.59%)이다. 이는 지배구조 개편과 장기투자 기반 형성에 부담으로 작용할 뿐 아니라 주주환원 확대 정책과도 충돌할 수 있다.

강성진 고려대 경제학과 교수는 "우리나라 상속세가 다른 나라에 비해 매우 높다는 것은 하루이틀 나온 이야기가 아니다"라며 "기업들이 상속세 부담을 줄이기 위해 일부러 기업 가치를 낮추거나 보유 지분 매각을 하는 등 지배구조 개선과는 멀어지기 때문에 하루빨리 개선해야 하는 과제"라고 짚었다.

■세 차례 유예된 가상자산 과세

디지털 자산 과세 역시 빠질 수 없는 과제다. 가상자산 소득과세는 지난 2020년 소득세법 개정을 통해 처음 제도화됐으나 투자자보호체계 부재와 과세 인프라 미흡 등으로 2023, 2025, 2027년까지 총 세차례 유예됐다. 오는 2027년 1월 시행을 앞두고 있으나 현재 세부 가이드라인은 마련하지 못한 상황이다. 관련 업계에서는 또다시 유예될 가능성이 제기되고 있다.

이효섭 자본시장연구원 연구위원은 "자본시장 활성화와 상생적 금융은 하나의 제도만 바뀐다고 해서 완성되는 것이 아니다"라며 "쌓여 있는 남은 과제를 어떻게 해결할 것인지 논의해야 할 시점"이라고 말했다. ''')

# =========================================================
# 셀 4: 실제 예시 테스트 (예시 2)
# =========================================================
# 예시 2: 동화/일상 (쉬운 단어 예상)
analyze_and_translate('''쿠팡 이용자인 A씨는 지난 6일 “개인 정보가 변경됐다”는 제목이 달린 e메일을 받았다. 발신자는 ‘noreply@coupang.com’으로 쿠팡의 이메일 주소 형식과 같았다. 메일은 A씨에게 첨부된 링크를 클릭하라고 안내했다. 이에 수상함을 느낀 A씨는 해당 메일을 경찰에 신고했다. 경찰이 조사해 보니 첨부된 링크는 ‘피싱 사이트’와 연결됐다. e메일 주소는 쿠팡 공식 메일 ‘noreply_@coupang.com’과 아주 조금 달랐다.

최근 쿠팡에서 3370만명에 달하는 계정 정보가 유출되자 이용자들의 불안감을 이용한 신종 피싱(금융기관 등을 사칭해 개인정보를 탈취하는 범법 행위)·스미싱(문자 메시지를 이용한 휴대전화 해킹) 범죄가 기승을 부리고 있다.


경찰청 ‘전기통신금융사기 통합대응단’은 7일 보도자료를 내 “쿠팡 개인정보 유출 상황을 악용한 새 유형의 스미싱·피싱 시나리오 신고가 접수되고 있다”고 밝혔다.

기존 ‘카드 배송 사칭’ 수법에 쿠팡을 끼워 넣는 신종 범죄도 최근 나타났다. 통합대응단이 수집한 사례를 보면 B씨는 지난 2일 “본인 명의 카드가 발급돼 배송 예정”이라는 전화를 받았다. B씨가 발급 신청을 한 적이 없다고 하자 “쿠팡 정보 유출로 문제가 발생한 것 같다”며 가짜 사고센터 전화번호를 알려줬다. 알려준 번호로 전화하니 범인들은 악성 앱이 깔렸는지 검사해야한다며 휴대전화에 원격 제어 앱을 설치하라고 유도했다.

쿠팡으로 주문한 물품 배송이 늦어지거나 빠질 수 있다는 내용의 문자 메시지를 보내, 특정 링크에 접속하도록 유도하는 방식도 있다. 경찰청 관계자는 “궁극적 목적은 개인 금융 정보 탈취로 보인다”고 말했다.


대응단은 과학기술정보통신부, 한국인터넷진흥원(KISA), 금융감독원 등 관계기관과 협력해 ‘피싱 의심번호’를 긴급 차단하는 등 대응에 나섰다.


대응단은 “출처를 알 수 없는 전화번호로 발송된 메시지나 인터넷 주소는 절대 누르지 말고 즉시 삭제해달라”고 당부했다. 또 “이미 노출된 개인정보를 활용해 접근 방식이 더 정교해질 수 있으니, 모르는 번호로 전화가 오면 보이스피싱 가능성을 의심해야 한다”고 말했다.''')

# =========================================================
# 셀 5: 실제 예시 테스트 (예시 3)
# =========================================================
# 예시 3: 질문자님의 데이터에 있을법한 문장
analyze_and_translate("""이어 우원식 국회의장과 조희대 대법원장, 김민석 국무총리 등 5부 요인을 초청해 오찬을 함께하고, 저녁에는 시민단체가 주관하는 장외행사에 참석합니다.""")

Device set to use cpu


▶ 입력 문장:
 내년 1월부터 배당소득 분리과세가 시행되면서 자본시장 과세체계가 큰 변곡점을 맞았다.

그동안 연간 금융소득 2000만원을 초과하면 종합과세로 묶이던 구조가 일부 손질되면서, 배당 투자 매력이 높아졌다는 평가가 나온다. 다만 전문가들은 이번 조치를 '세제 개편의 출발점'으로 보고 있다. 거래·보유·신규 자산을 아우르는 현행 과세체계가 국제 기준과 여전히 괴리를 보이고 있어, 후속 정비가 필수적이라는 이유에서다.

■금투세·거래세 정비 1순위

7일 정치권과 관련 당국에 따르면 지난 2일 통과된 조세특례제한법 개정안은 고배당 상장기업의 배당소득에 대해 분리과세를 적용하는 것이 핵심이다. 세율은 △2000만원 이하 14% △2000만원 초과~3억원 이하 20% △3억원 초과~50억원 이하 25% △50억원 초과 30%로 구간이 신설됐다. 적용 대상은 배당성향 40% 이상이거나, 배당성향 25% 이상이면서 전년 대비 10% 이상 배당을 늘린 상장사로, 내년 결산배당부터 바로 적용된다.

정부와 국회는 이번 조치가 기업의 배당정책을 개선하고, 주주환원 문화를 확산시키는 계기가 될 것으로 기대하고 있다. 그러나 전문가들은 "첫발을 뗐을 뿐"이라며, 구조적 문제를 해결하려면 추가적인 세제 개편이 필요하다고 지적한다.

우선 가장 시급한 과제로 꼽히는 것은 '금융투자소득세(금투세)'와 '증권거래세'의 체계 정비다. 기획재정부는 내년 1월부터 증권거래세율을 코스피 0.05%, 코스닥 0.20%로 각각 인상하기로 했다. 금투세 도입을 전제로 인하했던 증권거래세율을 금투세 도입 무산으로 인해 다시 올리는 것이다.

다만 거래 단계에서 부과되는 이 세금은 소득이 아닌 '거래 행위' 자체에 세금을 매기는 구조로, 주식시장 유동성 위축과 거래비용 증가 요인으로 작용할 수 있다는 비판이 나온다. 특히 경제협력개발기구(OECD) 주요국이 거래세를 폐지하거나 최소 수준으로 유지하고 있는 점을 고려하면, 앞으로 우리 세제가 '거래에 과세할지, 이익에 과세할지' 방향을 명확

Device set to use cpu


▶ 입력 문장:
쿠팡 이용자인 A씨는 지난 6일 “개인 정보가 변경됐다”는 제목이 달린 e메일을 받았다. 발신자는 ‘noreply@coupang.com’으로 쿠팡의 이메일 주소 형식과 같았다. 메일은 A씨에게 첨부된 링크를 클릭하라고 안내했다. 이에 수상함을 느낀 A씨는 해당 메일을 경찰에 신고했다. 경찰이 조사해 보니 첨부된 링크는 ‘피싱 사이트’와 연결됐다. e메일 주소는 쿠팡 공식 메일 ‘noreply_@coupang.com’과 아주 조금 달랐다.

최근 쿠팡에서 3370만명에 달하는 계정 정보가 유출되자 이용자들의 불안감을 이용한 신종 피싱(금융기관 등을 사칭해 개인정보를 탈취하는 범법 행위)·스미싱(문자 메시지를 이용한 휴대전화 해킹) 범죄가 기승을 부리고 있다.


경찰청 ‘전기통신금융사기 통합대응단’은 7일 보도자료를 내 “쿠팡 개인정보 유출 상황을 악용한 새 유형의 스미싱·피싱 시나리오 신고가 접수되고 있다”고 밝혔다.

기존 ‘카드 배송 사칭’ 수법에 쿠팡을 끼워 넣는 신종 범죄도 최근 나타났다. 통합대응단이 수집한 사례를 보면 B씨는 지난 2일 “본인 명의 카드가 발급돼 배송 예정”이라는 전화를 받았다. B씨가 발급 신청을 한 적이 없다고 하자 “쿠팡 정보 유출로 문제가 발생한 것 같다”며 가짜 사고센터 전화번호를 알려줬다. 알려준 번호로 전화하니 범인들은 악성 앱이 깔렸는지 검사해야한다며 휴대전화에 원격 제어 앱을 설치하라고 유도했다.

쿠팡으로 주문한 물품 배송이 늦어지거나 빠질 수 있다는 내용의 문자 메시지를 보내, 특정 링크에 접속하도록 유도하는 방식도 있다. 경찰청 관계자는 “궁극적 목적은 개인 금융 정보 탈취로 보인다”고 말했다.


대응단은 과학기술정보통신부, 한국인터넷진흥원(KISA), 금융감독원 등 관계기관과 협력해 ‘피싱 의심번호’를 긴급 차단하는 등 대응에 나섰다.


대응단은 “출처를 알 수 없는 전화번호로 발송된 메시지나 인터넷 주소는 절대 누르지 말고 즉시 삭제해달라”고 당부했다. 또 “이미 노출된 개인정보를 활용해 

Device set to use cpu


📌 어려운 단어 목록

1. 피싱 → phishing
2. 신종 → nuevas especies
3. 피싱(금융기관 → Phishing (instituciones financieras
4. 범법 → transgresión
5. 행위)·스미싱(문자 → Acto)·Smishing (texto
6. 기승 → equitación
7. 부리고 → haciéndolo
8. 전기통신금융사기 → Fraude financiero en telecomunicaciones
9. 통합대응단’ → Equipo de Respuesta Integrada
10. 스미싱·피싱 → phishing/phishing
11. 통합대응단 → Equipo de respuesta integrado
12. 유도하 → Judoha
13. 궁극적 → último
14. 관계기관 → Organizaciones relacionadas
15. 보이스피싱 → phishing de voz

=> 총 15개의 어려운 단어를 찾았습니다.
----------------------------------------

▶ 입력 문장:
이어 우원식 국회의장과 조희대 대법원장, 김민석 국무총리 등 5부 요인을 초청해 오찬을 함께하고, 저녁에는 시민단체가 주관하는 장외행사에 참석합니다.

----------------------------------------
📌 어려운 단어 목록

1. 우원식 → Woo Won Sik
2. 조희대 → Cho Hee Dae
3. 대법원장 → Presidente del Tribunal Supremo
4. 김민석 → Minseok Kim
5. 초청해 → invitarte
6. 오찬 → almuerzo
7. 주관하 → Anfitrión:
8. 장외행사 → Eventos fuera del sitio

=> 총 8개의 어려운 단어를 찾았습니다.
----------------------------------------

